In [2]:
import ollama
import time
import pandas as pd
from IPython.display import display 


In [3]:

MODELS = ['yi-coder:1.5b', 'qwen2.5-coder:1.5b', 'llama3.2:1b']


TEST_SUITE = [
   
    {"id": "PY_01", "lang": "Python", "type": "Logic", "name": "Off-by-one Error", 
     "code": "def get_items(arr):\n    # BUG: Index out of range\n    for i in range(len(arr) + 1):\n        print(arr[i])"},
    
    {"id": "PY_02", "lang": "Python", "type": "Security", "name": "SQL Injection", 
     "code": "def login(u, p):\n    # BUG: SQL Injection vulnerability\n    q = f\"SELECT * FROM users WHERE u='{u}' AND p='{p}'\"\n    db.execute(q)"},
    
    {"id": "PY_03", "lang": "Python", "type": "Runtime", "name": "Zero Division", 
     "code": "def average(nums):\n    total = sum(nums)\n    # BUG: Crash if nums is empty\n    return total / len(nums)"},
    
    {"id": "PY_04", "lang": "Python", "type": "Logic", "name": "Mutable Default Arg", 
     "code": "def append_to(num, target=[]):\n    # BUG: target retains value between calls\n    target.append(num)\n    return target"},

    # --- JAVA CASES ---
    {"id": "JAVA_01", "lang": "Java", "type": "Logic", "name": "String Comparison", 
     "code": "public boolean check(String s) {\n    // BUG: Using == instead of .equals\n    if (s == \"password\") return true;\n    return false;\n}"},
    
    {"id": "JAVA_02", "lang": "Java", "type": "Runtime", "name": "Null Pointer", 
     "code": "public int getLen(String s) {\n    // BUG: No null check\n    return s.length();\n}"},
    
    {"id": "JAVA_03", "lang": "Java", "type": "Logic", "name": "Infinite Loop", 
     "code": "public void loop() {\n    // BUG: Decrementing instead of incrementing\n    for (int i = 0; i < 10; i--) {\n        System.out.println(i);\n    }\n}"},
    
    {"id": "JAVA_04", "lang": "Java", "type": "Concurrency", "name": "Race Condition", 
     "code": "public class Counter {\n    private int count = 0;\n    // BUG: Not synchronized\n    public void increment() { count++; }\n}"},

    # --- C/C++ CASES ---
    {"id": "C_01", "lang": "C", "type": "Memory", "name": "Buffer Overflow", 
     "code": "void copy(char *s) {\n    char buf[5];\n    // BUG: Unsafe copy\n    strcpy(buf, s);\n}"},
    
    {"id": "C_02", "lang": "C", "type": "Memory", "name": "Memory Leak", 
     "code": "void process() {\n    int *p = malloc(sizeof(int)*10);\n    p[0] = 1;\n    // BUG: No free(p)\n}"},
    
    {"id": "C_03", "lang": "C", "type": "Logic", "name": "Integer Overflow", 
     "code": "int add(int a, int b) {\n    // BUG: Can overflow if a+b > MAX_INT\n    return a + b;\n}"},
    
    {"id": "C_04", "lang": "C", "type": "Pointer", "name": "Use After Free", 
     "code": "void func() {\n    char *p = malloc(10);\n    free(p);\n    // BUG: Accessing freed memory\n    *p = 'a';\n}"}
]



In [ ]:
# --- 3. HÀM CHẠY BENCHMARK ---
def run_benchmark():
    results = []
    total_tasks = len(TEST_SUITE) * len(MODELS)
    current_task = 0

    print(f" BENCHMARK: {len(TEST_SUITE)} bài test x {len(MODELS)} model")
    print("-" * 60)

    for case in TEST_SUITE:
        print(f" [{case['lang']}] {case['name']}...")
        
        for model in MODELS:
            current_task += 1
            print(f"   [{current_task}/{total_tasks}] Running {model}...", end=" ", flush=True)
            
            try:
                start_time = time.time()
                
                
                prompt = f"Fix this {case['lang']} code. Return ONLY the fixed code block:\n```\n{case['code']}\n```"
                
                
                response = ollama.chat(model=model, keep_alive=0, messages=[
                    {'role': 'user', 'content': prompt}
                ])
                
                duration = round(time.time() - start_time, 2)
                output = response['message']['content']
                
                print(f"Xong ({duration}s)")
                
                results.append({
                    "Model": model,
                    "Ngôn ngữ": case["lang"],
                    "Loại lỗi": case["type"],
                    "Bài toán": case["name"],
                    "Thời gian (s)": duration,
                    "Code Sửa": output 
                })
                
            except Exception as e:
                print(f"❌ Lỗi: {e}")
                results.append({
                    "Model": model,
                    "Ngôn ngữ": case["lang"],
                    "Loại lỗi": case["type"],
                    "Bài toán": case["name"],
                    "Thời gian (s)": 0,
                    "Code Sửa": str(e)
                })

    
    print("\n" + "="*60)
    print(" KẾT QUẢ CHI TIẾT")
    df = pd.DataFrame(results)
    
 
    display(df) 

    print("\n" + "="*60)
    print(" TÓM TẮT TỐC ĐỘ TRUNG BÌNH (Giây)")
    
  
    summary_df = df.pivot_table(index="Model", columns="Ngôn ngữ", values="Thời gian (s)", aggfunc="mean")
    
  
    display(summary_df)

   
    print("\n" + "="*60)
    print(" BIỂU ĐỒ SO SÁNH")
    try:
        summary_df.plot(kind='bar', figsize=(10, 6), title="Tốc độ xử lý trung bình theo Ngôn ngữ (Thấp hơn là tốt hơn)")
    except ImportError:
        print("Cần cài matplotlib để vẽ biểu đồ: pip install matplotlib")



In [ ]:
# Chạy hàm
run_benchmark()

🔥 BẮT ĐẦU BENCHMARK: 12 bài test x 3 model
------------------------------------------------------------
📂 Đang xử lý: [Python] Off-by-one Error...
   [1/36] Running yi-coder:1.5b... ✅ Xong (16.57s)
   [2/36] Running qwen2.5-coder:1.5b... ✅ Xong (9.09s)
   [3/36] Running llama3.2:1b... ✅ Xong (6.76s)
📂 Đang xử lý: [Python] SQL Injection...
   [4/36] Running yi-coder:1.5b... ✅ Xong (35.31s)
   [5/36] Running qwen2.5-coder:1.5b... ✅ Xong (9.06s)
   [6/36] Running llama3.2:1b... ✅ Xong (14.75s)
📂 Đang xử lý: [Python] Zero Division...
   [7/36] Running yi-coder:1.5b... ✅ Xong (11.31s)
   [8/36] Running qwen2.5-coder:1.5b... ✅ Xong (16.09s)
   [9/36] Running llama3.2:1b... ✅ Xong (14.53s)
📂 Đang xử lý: [Python] Mutable Default Arg...
   [10/36] Running yi-coder:1.5b... ✅ Xong (16.87s)
   [11/36] Running qwen2.5-coder:1.5b... ✅ Xong (6.27s)
   [12/36] Running llama3.2:1b... ✅ Xong (7.33s)
📂 Đang xử lý: [Java] String Comparison...
   [13/36] Running yi-coder:1.5b... ✅ Xong (12.46s)
   [14/36] 